In [2]:
import onnxruntime as ort
import numpy as np

###  ONNX Inference

In [1]:
onnx_model_path = "/Users/bob/docs/ByteDance/PNN/converter/load_fail.onnx"

In [3]:
input_path = "/Users/bob/Downloads/windows_ci_report/python_test_report/tmp/20220622-14-29-41/convolution_2d_0_input.npz"

In [8]:
onnx_output_path = "/Users/bob/Downloads/windows_ci_report/python_test_report/tmp/20220622-14-29-41/convolution_2d_0_output.npz"

In [5]:
sess = ort.InferenceSession(onnx_model_path)

#### Read Input and to binary

In [4]:
inputs = {}
with np.load(input_path) as data:
    for  k in data.keys():
        inputs[k] = data[k]

In [6]:
inputs['input'].shape

(2, 28, 12, 2)

In [7]:
inputs['input'].tofile("/Users/bob/Downloads/windows_ci_report/python_test_report/tmp/20220622-14-29-41/convolution_2d_0_input.bin")

#### Read output and to binary

In [17]:
onnx_infered_outputs = {}
with np.load(onnx_output_path) as data:
    for  k in data.keys():
        onnx_infered_outputs[k] = data[k]

In [21]:
onnx_infered_outputs['reduce_mean'].tofile("/Users/bob/Downloads/reduce_mean_3_input/reduce_mean_3_output.bin")

#### Run ONNX

In [10]:
outputs = sess.run(None, inputs)

In [11]:
outputs[0].flatten()[:10]

array([0.44793013, 0.5031735 , 0.47086844, 0.42500693, 0.49454945,
       0.46459344, 0.4326595 , 0.47928378, 0.6065165 , 0.5034367 ],
      dtype=float32)

In [12]:
outputs[0].shape

(29, 40, 1, 15)

### Test P

### PNN API

In [140]:
np.where(abs(test_p - 1.1426839) < 1e-5)

(array([ 0,  2,  3,  8, 10]),
 array([ 0, 18,  2,  3, 17]),
 array([ 0,  3, 12,  3,  3]),
 array([ 4,  2,  1, 14, 10]))

In [144]:
test_p[10,17,3,10]

1.1426804

### PNN Inference

In [12]:
import pnn

In [13]:
model_path = "/Users/bob/Downloads/reduce_mean_3_input/reduce_mean_3_gpu.pnn"

In [14]:
config = pnn.engine_config()
config.devices = [pnn.cpu_device()]  # now don't support inplace assign, eg: append
# need to learn pybind11 to handle this
engine = pnn.engine(config)
status = engine.load_model(model_path)

ERROR [/Users/bob/code/ByteDance/PNN/src/core/Engine.cpp:81::LoadGraph] Partition failed caused by: Unsupported


In [211]:
assert status == pnn.STATUS.OK

pnn_inputs = engine.mutable_inputs()  # type: list

infer_result = {}


def after_callback(outputs, metric: pnn.op_metric):
    # get executor/node name -> metric.name
    for o in outputs:
        infer_result[o.name] = o.data()
    return True

In [212]:
with np.load(input_path) as values:
    for i in pnn_inputs:
        i.assign(values[i.name])

In [213]:
status = engine.run(None, after_callback)
assert status == pnn.STATUS.OK

outputs = engine.get_outputs()

print(outputs[0].data().shape)
o = outputs[0].data().flatten()
print(o[:10])
print(o.dtype)

print(infer_result["matmul"].shape)
print(infer_result["matmul"].flatten()[:10])

(2, 16)
[1.         1.         1.         0.74791116 1.         1.
 1.         1.         1.         1.        ]
float32
(2, 16)
[1.         1.         1.         0.74791116 1.         1.
 1.         1.         1.         1.        ]


In [9]:
import numpy as np

In [13]:
tmp_path = "/Users/bob/Downloads/windows_ci_report/python_test_report/tmp/20220622-14-29-41/convolution_2d_0_onnx_output.npz"

In [14]:
a = {}

In [15]:
with np.load(tmp_path) as data:
    for  k in data.keys():
        a[k] = data[k]

In [17]:
a['convolution'].tofile("/Users/bob/Downloads/windows_ci_report/python_test_report/tmp/20220622-14-29-41/convolution_2d_0_output.bin")

In [1]:
import os

In [4]:
for path, dirnames, filenames in os.walk('/Users/bob/code/ByteDance/PNN/tools/converter'):
    # print path to all subdirectories first.
    for subdirname in dirnames:
        print(path,subdirname)

/Users/bob/code/ByteDance/PNN/tools/converter test
/Users/bob/code/ByteDance/PNN/tools/converter src
/Users/bob/code/ByteDance/PNN/tools/converter/test onnx
/Users/bob/code/ByteDance/PNN/tools/converter/test test_data
/Users/bob/code/ByteDance/PNN/tools/converter/test tensorflow
/Users/bob/code/ByteDance/PNN/tools/converter/test/test_data onnx
/Users/bob/code/ByteDance/PNN/tools/converter/test/test_data tensorflow
/Users/bob/code/ByteDance/PNN/tools/converter/test/test_data/onnx mobilenetv2
/Users/bob/code/ByteDance/PNN/tools/converter/test/test_data/onnx deepfm
/Users/bob/code/ByteDance/PNN/tools/converter/test/test_data/tensorflow mobilenetv2
/Users/bob/code/ByteDance/PNN/tools/converter/test/test_data/tensorflow deepfm
/Users/bob/code/ByteDance/PNN/tools/converter/src core
/Users/bob/code/ByteDance/PNN/tools/converter/src onnx
/Users/bob/code/ByteDance/PNN/tools/converter/src common
/Users/bob/code/ByteDance/PNN/tools/converter/src tensorflow
/Users/bob/code/ByteDance/PNN/tools/conv

In [18]:
import tensorflow as tf

In [19]:
input_shape = (4, 28, 28, 3)
x = tf.random.normal(input_shape)
y = tf.keras.layers.Conv2D(
2, 3, activation='relu', input_shape=input_shape[1:])(x)

2022-04-19 20:41:16.060005: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-04-19 20:41:16.062321: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [29]:
import numpy as np

In [30]:
a = np.random.rand(2,1,4)

In [33]:
b = np.random.rand(4)

In [34]:
a - b

array([[[-0.16892279,  0.15003531, -0.57425805,  0.3597487 ]],

       [[-0.75622075, -0.08833513, -0.76440729, -0.05332514]]])

In [36]:
c = a-b

In [37]:
c.shape

(2, 1, 4)

### Infer Qualitai

In [35]:
import numpy as np
import onnx
import onnxruntime as ort
from onnx.helper import make_tensor, make_model, make_graph
from onnx import ValueInfoProto

In [23]:
save_prefix = "/Users/bob/docs/ByteDance/PNN/converter/quality/"

In [29]:
model_path = "/Users/bob/docs/ByteDance/PNN/converter/quality/load_fail.onnx"

In [15]:
data = np.random.uniform(size = [1,3,640,640]).astype(np.float32)

In [17]:
sess = ort.InferenceSession(model_path)

In [19]:
outputs  = sess.run(None, {"images" : data})

In [24]:
outputs[0].tofile(save_prefix + "onnx_output.bin")

In [25]:
data.tofile(save_prefix + "onnx_input.bin")

In [27]:
outputs[0].flatten()[0]

0.9429071

In [28]:
outputs  = sess.run(None, {"images" : data})

InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Invalid Output Name:Mul_142

In [83]:
onnx_model = onnx.load(model_path)

In [84]:
def extend_model_outputs(model, outputs=None):
    """
    Extend model's outputs
    :param model: *ONNX* model
    :param outputs: new outputs -> str list
    :return: modified model
    """
    if outputs is None:
        raise ValueError("Outputs cannot be None.")

    if not isinstance(outputs, list):
        outputs = [outputs]
    original_outputs = [o.name for o in model.graph.output]  # type: list

    var_out = []
    for out in outputs:
        if out in original_outputs:
            continue
        value_info = ValueInfoProto()
        value_info.name = out
        var_out.append(value_info)
    # extend original graph outputs
    var_out.extend(list(model.graph.output))
    graph = make_graph(model.graph.node, model.graph.name, model.graph.input,
                       var_out, model.graph.initializer)
    onnx_model = make_model(graph)
    onnx_model.ir_version = model.ir_version
    onnx_model.producer_name = model.producer_name
    onnx_model.producer_version = model.producer_version
    onnx_model.domain = model.domain
    onnx_model.model_version = model.model_version
    onnx_model.doc_string = model.doc_string
    if len(model.metadata_props) > 0:
        values = {p.key: p.value for p in model.metadata_props}
        onnx.helper.set_model_props(onnx_model, values)

    if len(onnx_model.graph.input) != len(model.graph.input):
        raise RuntimeError("Input mismatch {} != {}".format(
            len(onnx_model.input), len(model.input)))

    # fix opset import
    del onnx_model.opset_import[:]
    for oimp in model.opset_import:
        op_set = onnx_model.opset_import.add()
        op_set.domain = oimp.domain
        op_set.version = oimp.version
    return onnx_model

In [85]:
new_model = extend_model_outputs(onnx_model,["614","616"])

In [86]:
onnx.save(new_model, save_prefix + "new_model.onnx")

In [41]:
new_sess = ort.InferenceSession(save_prefix + "new_model.onnx")

In [53]:
new_outputs  = new_sess.run(["output","642","644"], {"images" : data})

In [56]:
new_outputs[1].tofile(save_prefix + "upsample_input.bin")

In [57]:
new_outputs[2].tofile(save_prefix + "upsample_output.bin")

In [58]:
upsample_input = new_outputs[1]

In [64]:
upsample_input.flatten()[:10]

array([ 0.78252816,  0.79711086,  1.1949968 ,  0.952159  ,  0.7991512 ,
        0.7115123 ,  0.53460616,  0.68280643, -0.0542749 ,  0.3403077 ],
      dtype=float32)

In [62]:
upsample_output = new_outputs[2]

In [72]:
upsample_output.flatten()[:16]

array([0.78252816, 0.7898195 , 0.79711086, 0.9960538 , 1.1949968 ,
       1.0735779 , 0.952159  , 0.87565506, 0.7991512 , 0.75533175,
       0.7115123 , 0.6230593 , 0.53460616, 0.6087063 , 0.68280643,
       0.31426576], dtype=float32)

In [ ]:
def nchw2nc4hw4(nchw_data):
    shape = nchw_data.shape
    original_c = shape[1]
    c4 = (shape[1] // 4) + 1
    shape[1] = c4
    shape.append(4)
    c4array = np.zeros(shape)
    for b in range(shape[0]):
        for i in range(c4):
            c = c4 * 4
            if c < original_c:
                c4array[b,c:c+4,]

In [69]:
result = new_outputs[0]

In [82]:
result.flatten()[1]

0.6733867

In [73]:
test_output = np.fromfile(save_prefix + "upsample_output.bin",dtype=np.float32)

In [81]:
test_output[11564]

4.0294362e-05

### Fix PNN